In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet34
from torchvision.datasets import ImageFolder
import json
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
model = resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
model.eval()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
with open("../TestDataSet/labels_list.json", "r") as f:
    idx_map = json.load(f)

In [4]:
# Transformation: matches the ImageNet normalization used by ResNet models

mean_norms = [0.485, 0.456, 0.406]
std_norms = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])

In [5]:
dataset = ImageFolder(root="../TestDataSet", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
def evaluate_topk(model, dataloader, k=(1, 5)):
    model.eval()
    total = 0
    top1_correct = 0
    top5_correct = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images = images.cuda()
            labels = labels.cuda()

            outputs = model(images)
            _, topk_preds = outputs.topk(max(k), dim=1, largest=True, sorted=True)

            top1_correct += (topk_preds[:, 0] == labels).sum().item()
            top5_correct += sum([labels[i] in topk_preds[i] for i in range(len(labels))])
            total += labels.size(0)

    top1_acc = top1_correct / total * 100
    top5_acc = top5_correct / total * 100

    return top1_acc, top5_acc


In [ ]:
top1, top5 = evaluate_topk(model, dataloader)
print(f"Top-1 Accuracy: {top1:.2f}%")
print(f"Top-5 Accuracy: {top5:.2f}%")